In [1]:
#Libraries / Functions
import numpy as np
import pandas as pd
import requests
import json
from urllib.request import urlopen
import numpy as np
import pandas as pd
import requests
import json
#import pytrends
#from pytrends.request import TrendReq
import time
from datetime import datetime
from dateutil.relativedelta import *
startTime = time.time()
#pytrend = TrendReq(hl='en-GB', tz=360)

In [2]:
x1 = 476000
x2 = 480000
count = 0

while count<8:

  movielist=[]
  for i in range(x1,x2):
        url="https://api.themoviedb.org/3/movie/"+str(i)+"?api_key=4568df78fb309238e68974a24013b626"
        try:
            JSONContent = urlopen(url)
        except:
            #If there's no such movie, then go to the next loop
            continue
        detaildata = json.loads(JSONContent.read())
        movielist.append(detaildata)  #Add the details to the list

     
  movieData=pd.json_normalize(movielist)  #Convert raw JSON data in list to Dataframe

  
  movieData = movieData[movieData['original_language'] == 'en'] #Drop rows that do not have English as og language

  #Drop rows that have no tagline/imdb_id
  movieData=movieData[movieData['tagline'].notnull()]
  movieData=movieData[movieData['imdb_id'].notnull()]
 
  movieData=movieData[movieData['adult']!=True]  #Drop adult films
  movieData.to_csv('movies/' + str(x1) + '-' + str(x2) + '.csv')
  x1 = x1 + 2000
  x2 = x2 + 2000
  count = count + 1

FileNotFoundError: [Errno 2] No such file or directory: 'movies/476000-480000.csv'

In [ ]:
#Combine the Dataframes into 1
titleData8=pd.read_csv('movies/476000-480000.csv')
titleData7=pd.read_csv('movies/480000-484000.csv')
titleData1=pd.read_csv('movies/484000-486000.csv')
titleData2=pd.read_csv('movies/486000-488000.csv')
titleData3=pd.read_csv('movies/488000-490000.csv')
titleData4=pd.read_csv('movies/490000-492000.csv')
titleData5=pd.read_csv('movies/492000-494013.csv')
titleData6=pd.read_csv('movies/494013-496000.csv')

comb=[titleData8,titleData7,titleData1,titleData2,titleData3,titleData4,titleData5,titleData6]
final=pd.concat(comb)

final

In [ ]:
oscarData=pd.read_csv("oscaraward.csv") #Drop oscar nomination to individual that did not work on any films

oscarData=oscarData[oscarData['film'].notnull()]#Drop movies older than 2010

oscarData=oscarData[oscarData['year_film']>=2010]#Create a new Dataframe with film's year of release and name

cleanOscarData=pd.DataFrame(oscarData[['year_film','film']])#Count how many times film appeared in the Dataframe and append it to a new row called freq

cleanOscarData['freq']=cleanOscarData.groupby('film').transform('count') #Delete all duplicates rows

cleanOscarData.drop_duplicates('film',keep='first',inplace=True)

cleanOscarData.to_csv("cleanoscar.csv")

In [ ]:
movielist=[]
for i in range(len(cleanOscarData['film'])):
    movieName=cleanOscarData['film'].values[i] #Ignore weird ascii characters like Ã,©
    
    movieName = movieName.encode('ascii', 'ignore').decode('ascii') #TMDB uses "+" instead of space for separators, so convert all spaces to "+""
    
    movieName=movieName.replace(" ","+") #Get year of release
    
    oscarYear=cleanOscarData['year_film'].values[i]
   
    url = "https://api.themoviedb.org/3/search/movie?api_key=4568df78fb309238e68974a24013b626&query="+movieName  #Query for movie id
    JSONContent = urlopen(url)
    data = json.loads(JSONContent.read())
    
    #Enter the "results" array in data
    for i in range(len(data['results'])):
        results=data['results'][i]
        date=results['release_date']
        date=date.split("-")[0]
        #Due to number of movies with similar name,
        #Compare year of release to Oscar nomination year
        #Limitation: If there's another movie with the similar name and same year of release, it might grab the wrong movie
        if (str(oscarYear) == date.strip()):
            movieID=results['id']
            break

    #Query for movie details
    url="https://api.themoviedb.org/3/movie/"+str(movieID)+"?api_key=4568df78fb309238e68974a24013b626"
    try:
        JSONContent = urlopen(url)
    except:
        pass
    detaildata = json.loads(JSONContent.read())
    movielist.append(detaildata)
   
    movieData=pd.json_normalize(movielist) 
    
#Export Dataframe
movieData.to_csv('omoviedb.csv')


In [ ]:
cleanOscarData=pd.read_csv("cleanoscar.csv")
movieData = pd.read_csv("omoviedb.csv")
#Get the release date of the movie
#Match film name is cleanOscarData to film name in movieData
cleanOscarData['release_date'] = cleanOscarData.film.map(movieData.set_index('title')['release_date'])
#Remove rows that do no have a release date
cleanOscarData=cleanOscarData[cleanOscarData['release_date'].notnull()]
cleanOscarData

In [ ]:
#Collect data from Google Trend
dataset = []
for x in range(400,458):
    #Convert date string to Datetime object
     date= datetime.strptime(cleanOscarData['release_date'][x], '%Y-%m-%d').date()
    #Add a month to the Datetime
     date2 = date+relativedelta(months=+1)
     timeframeDate = str(date) + " "+str(date2)
    #Keyword = Film name
     keywords = [cleanOscarData['film'][x]]
     pytrend.build_payload(
     kw_list=keywords,
     cat=0,
    #Timeframe = a month after movie release
     timeframe=timeframeDate,
    #Location = US
     geo='US')
     data = pytrend.interest_over_time()
     if not data.empty:
          data = data.drop(labels=['isPartial'],axis='columns')
          dataset.append(data)

result = pd.concat(dataset, axis=1)

executionTime = (time.time() - startTime)
print('Execution time in sec.: ' + str(executionTime))


In [ ]:
#Export Google Data
result.to_csv("googleTrends/457.csv")

In [ ]:
test=pd.read_csv('googleTrends/457.csv')
#Set header as "date"
test.set_index(test['date'],inplace=True)
test=test.transpose()
#Remove duplicate header
test=test[1:]
test